In [1]:
!pip3 install torch torchvision torchaudio

In [4]:
import torch 
torch.cuda.is_available()
!pip install nibabel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.8 MB/s eta 0:00:008.0 MB/s eta 0:00:01


In [10]:
import os
import numpy as np
import nibabel as nib
from PIL import Image
from torch.utils.data import Dataset, DataLoader

class DeepLesionDataset(Dataset):
    def __init__(self, data_dir, train_csv, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        
        # Read the CSV file and store the information
        self.info = pd.read_csv(train_csv)
        
        # Get the list of image files
        self.nifti_files = [f for f in os.listdir(os.path.join(data_dir, 'Images_nifti')) if f.endswith('.nii.gz')]
        self.png_files = [f for f in os.listdir(os.path.join(data_dir, 'Images_png')) if f.endswith('.png')]
    
    def __len__(self):
        return len(self.nifti_files)
    
    def __getitem__(self, index):
        # Load NIfTI image
        nifti_path = os.path.join(self.data_dir, 'Images_nifti', self.nifti_files[index])
        nifti_image = nib.load(nifti_path).get_fdata()
        
        # Load PNG image
        png_path = os.path.join(self.data_dir, 'Images_png', self.png_files[index])
        png_image = Image.open(png_path)
        
        # Apply transforms if provided
        if self.transform:
            nifti_image = self.transform(nifti_image)
            png_image = self.transform(png_image)
        
        # Get the corresponding label from the CSV file
        label = self.info.loc[index, 'label']
        
        return nifti_image, png_image, label

In [11]:
import torch
import torch.nn as nn
import torchvision.models as models

class UniversalLesionDetector(nn.Module):
    def __init__(self, num_classes):
        super(UniversalLesionDetector, self).__init__()
        self.num_classes = num_classes
        
        # Backbone network (VGG-16)
        vgg = models.vgg16(pretrained=True)
        self.features = nn.Sequential(*list(vgg.features.children())[:-2])
        
        # Region Proposal Network (RPN)
        self.rpn = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 1),
            nn.ReLU(inplace=True)
        )
        self.rpn_cls = nn.Conv2d(512, 18, 1)
        self.rpn_reg = nn.Conv2d(512, 36, 1)
        
        # RoI Pooling
        self.roi_pool = nn.AdaptiveMaxPool2d(7)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 5, padding=0),
            nn.ReLU(inplace=True),
            nn.Flatten(),
            nn.Linear(512, num_classes),
            nn.Linear(512, num_classes * 4)
        )
    
    def forward(self, x):
        # Extract features
        features = self.features(x)
        
        # Region Proposal Network
        rpn_output = self.rpn(features)
        rpn_cls_scores = self.rpn_cls(rpn_output)
        rpn_bbox_pred = self.rpn_reg(rpn_output)
        
        # Generate proposals
        # ... (code for generating proposals from RPN outputs)
        
        # RoI Pooling
        roi_features = self.roi_pool(features, proposals)
        
        # Classifier
        cls_scores = self.classifier[:5](roi_features)
        cls_scores = cls_scores.view(cls_scores.size(0), -1)
        bbox_pred = self.classifier[5:](roi_features)
        bbox_pred = bbox_pred.view(bbox_pred.size(0), -1, 4)
        
        return cls_scores, bbox_pred

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd 



# Set the paths to the DeepLesion dataset
data_dir = './'
train_csv = 'DL_info.csv'  # CSV file with training data information

# Set the hyperparameters
num_classes = 2  # Number of classes (lesion vs. non-lesion)
batch_size = 4
num_epochs = 10
learning_rate = 0.001

# Create the dataset and data loader
train_dataset = DeepLesionDataset(data_dir, train_csv, transform=None)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create the model, loss function, and optimizer
model = UniversalLesionDetector(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'universal_lesion_detector.pth')

/home/nevdread/miniconda3/envs/phas/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nevdread/miniconda3/envs/phas/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/nevdread/.cache/torch/hub/checkpoints/vgg16-397923af.pth
4.4%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopu